In [1]:
from astropy.io import fits
import astropy.wcs as wcs
import astropy.units as u
from astropy.convolution import convolve_fft
from astropy.stats import mad_std
import numpy as np
from radio_beam import Beam
from reproject import reproject_interp
from matplotlib import pyplot as plt
from scipy.ndimage import binary_dilation
from astropy.modeling import models, fitting
from astropy.table import Table, vstack 
from glob import glob 
from synphot import SpectralElement, units
import os

import warnings 
warnings.filterwarnings('ignore')

# Loading files

In [2]:
def get_hdu(rootdir, filename, hdu_id=0, return_filename=False):
    filename_full = glob(rootdir+filename)[0]
    if hdu_id == 'all':
        hdu = fits.open(filename_full)
    else:
        hdu = fits.open(filename_full)[hdu_id]
    print(filename_full)

    if return_filename: 
        return(hdu, filename_full)
    else:  
        return(hdu)

galaxy = 'ngc3351'
galaxy_muse = galaxy
rootdir = '/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/%s/' %galaxy
rootdir_bp = '/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_misc/hst_filters/' 
outputdir = 'hst_contsub_f547m/'

narrowband_filter = 'f658n' # newer image
instrument_f555w  = 'uvis'
instrument_f65Xn = 'uvis'
instrument_f814w = 'uvis'

hdu_muse_stars  = get_hdu(rootdir, 'muse/%s_starmask.fits' %galaxy_muse.upper())
hdu_muse_neb    = get_hdu(rootdir, 'muse/%s_nebmask.fits' %galaxy_muse.upper())
hdu_muse        = get_hdu(rootdir, 'muse/%s*_MAPS.fits' %galaxy_muse.upper(), 'all')

hdu_hst_f555w   = get_hdu(rootdir, 'hst/%s*_%s_*f547m*.fits' %(galaxy, instrument_f555w))
hdu_hst_f65Xn   = get_hdu(rootdir, 'hst/%s*_%s_*%s*.fits' %(galaxy, instrument_f65Xn, narrowband_filter))
hdu_hst_f814w   = get_hdu(rootdir, 'hst/%s*_%s_*f814w*.fits' %(galaxy, instrument_f814w))

hst_res  = 0.07 * u.arcsec
_, file_muse_f65Xn = get_hdu(rootdir, 'muse/%s*_%s_*%s.fits' %(galaxy_muse.upper(), instrument_f65Xn.upper(), narrowband_filter.upper()), return_filename=True)
muse_res = np.float32(file_muse_f65Xn.split('asec')[0].split('-')[-1]) * u.arcsec

/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc3351/muse/NGC3351_starmask.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc3351/muse/NGC3351_nebmask.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc3351/muse/NGC3351-1.05asec_MAPS.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc3351/hst/ngc3351_uvis_f547m_exp_drc_sci.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc3351/hst/ngc3351_uvis_f658n_exp_drc_sci.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc3351/hst/ngc3351_uvis_f814w_exp_drc_sci.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc3351/muse/NGC3351-1.05asec_UVIS_F658N.fits


In [3]:
def make_paths(rootdir):
    
    print('[Info] Outputing to the following:')
    print(rootdir+outputdir)

    if not os.path.isdir(rootdir+outputdir):
        os.mkdir(rootdir+outputdir)  
    if not os.path.isdir(rootdir+outputdir+'/figs'):
        os.mkdir(rootdir+outputdir+'/figs')
    os.system('rm -rf '+rootdir+outputdir+'/*.fits')

make_paths(rootdir)

[Info] Outputing to the following:
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc3351/hst_contsub_f547m/


In [4]:
def get_bandpassinfo(files):

    files.sort()

    bp = {}
    for file in files:

        print(file)

        area = 45238.93416 * units.AREA  # HST
        bp_ = SpectralElement.from_file(file)
        name = file.split('/')[-1].split('.dat')[0].replace('HST_', '').replace('.F', '_F')
        name = name.replace('WFC_', '')
        name = name.replace('WFC3_', '')
        name = name.replace('UVIS1', 'UVIS')
        name = name.replace('UVIS2', 'UVIS')

        bp[name] = {'equivwidth': bp_.equivwidth().value, 
                    'integrate': bp_.integrate().value, 
                    'rmswidth': bp_.rmswidth().value, 
                    'photbw': bp_.photbw().value, 
                    'fwhm': bp_.fwhm().value, 
                    'rectwidth': bp_.rectwidth().value, 
                    'pivot': bp_.pivot().value, 
                    'unit_response': bp_.unit_response(area).value}  
    
    return(bp)

files_bp = glob('%s*.dat' %rootdir_bp)
bp = get_bandpassinfo(files_bp)
bp.keys()

/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_misc/hst_filters/HST_ACS_WFC.F550M.dat
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_misc/hst_filters/HST_ACS_WFC.F555W.dat
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_misc/hst_filters/HST_ACS_WFC.F658N.dat
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_misc/hst_filters/HST_ACS_WFC.F814W.dat
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_misc/hst_filters/HST_WFC3_UVIS1.F555W.dat
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_misc/hst_filters/HST_WFC3_UVIS1.F657N.dat
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_misc/hst_filters/HST_WFC3_UVIS1.F658N.dat
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_misc/hst_filters/HST_WFC3_UVIS1.F814W.dat
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_misc/hst_filters/HST_WFC3_UVIS2.F547M.dat


dict_keys(['ACS_F550M', 'ACS_F555W', 'ACS_F658N', 'ACS_F814W', 'UVIS_F555W', 'UVIS_F657N', 'UVIS_F658N', 'UVIS_F814W', 'UVIS_F547M'])

# Correct units 
HST -> electron/s to ergcm2sA 
MUSE - > Jy to ergcm2sA

In [5]:
def get_nanzeros(hdu):
    hdu.data[hdu.data == 0] = np.nan
    return(hdu)

hdu_hst_f555w = get_nanzeros(hdu_hst_f555w)
hdu_hst_f65Xn = get_nanzeros(hdu_hst_f65Xn)
hdu_hst_f814w = get_nanzeros(hdu_hst_f814w)

In [6]:
def get_electrons_2_ergcm2sA(hdu, photflam=None, photplam=None, photbw=None):

    data = hdu.data.copy()

    if photflam == None: 

        # Get the necessary header keywords for scaling and conversion
        photflam = hdu.header['PHOTFLAM']
        photplam = hdu.header['PHOTPLAM']
        photbw = hdu.header['PHOTBW']
    
    # Scale the data using photflam and photbw
    data_conv = data * photflam

    hdu.data = np.array(data_conv, dtype=np.float32) *1e20
    hdu.header['BUNIT'] = ('erg/s/cm2/A/pixel', '1e-20 erg/s/cm2/A')

    return(hdu)

hdu_hst_f555w = get_electrons_2_ergcm2sA(hdu_hst_f555w, bp['%s_%s' %(instrument_f555w.upper(), 'F547M')]['unit_response'])
hdu_hst_f65Xn = get_electrons_2_ergcm2sA(hdu_hst_f65Xn, bp['%s_%s' %(instrument_f65Xn.upper(), narrowband_filter.upper())]['unit_response'])
hdu_hst_f814w = get_electrons_2_ergcm2sA(hdu_hst_f814w, bp['%s_%s' %(instrument_f814w.upper(), 'F814W')]['unit_response'])

# Continuum subtraction

In [7]:
def get_contsub(hdu_halpha, hdu_cont1, hdu_cont2, 
                photplam_halpha=None, photplam_cont1=None, photplam_cont2=None):

    if photplam_halpha == None:
        photplam_halpha = hdu_halpha.header['PHOTPLAM']
        photplam_cont1 = hdu_cont1.header['PHOTPLAM']
        photplam_cont2 = hdu_cont2.header['PHOTPLAM']

    weight_cont1 = abs(photplam_cont2 - photplam_halpha) / abs(photplam_cont1 - photplam_cont2)
    weight_cont2 = abs(photplam_cont1 - photplam_halpha) / abs(photplam_cont1 - photplam_cont2)

    coef_cont1 = weight_cont1
    coef_cont2 = weight_cont2

    data_cont1 = hdu_cont1.data
    data_cont2 = hdu_cont2.data

    data_cont1[data_cont1<=0] = np.nan
    data_cont2[data_cont2<=0] = np.nan

    data_cont1 = np.log10(data_cont1)
    data_cont2 = np.log10(data_cont2)

    hdu_cont1.data = data_cont1 * coef_cont1
    hdu_cont2.data = data_cont2 * coef_cont2

    data_cont = 10**(hdu_cont1.data + hdu_cont2.data)
    data_cont[np.isnan(data_cont)] = 0

    hdu_halpha_cont = hdu_halpha.copy()
    hdu_halpha_contsub = hdu_halpha.copy()

    hdu_halpha_cont.data = data_cont
    hdu_halpha_contsub.data = hdu_halpha.data - data_cont

    hdu_halpha_cont.data = np.array(hdu_halpha_cont.data, dtype=np.float32)
    hdu_halpha_contsub.data = np.array(hdu_halpha_contsub.data, dtype=np.float32)

    return(hdu_halpha_contsub, hdu_halpha_cont) 

hdu_hst_halpha, hdu_hst_cont = get_contsub(hdu_hst_f65Xn.copy(), 
                                hdu_hst_f555w.copy(), 
                                hdu_hst_f814w.copy(), 
                                bp['%s_%s' %(instrument_f65Xn.upper(), narrowband_filter.upper())]['pivot'], 
                                bp['%s_%s' %(instrument_f555w.upper(), 'F547M')]['pivot'], 
                                bp['%s_%s' %(instrument_f814w.upper(), 'F814W')]['pivot'])                                                          

Correct units 
ergcm2sA to ergcm2s

In [8]:
def get_ergcm2sA_2_ergcm2s(hdu, photbw):

    data = hdu.data.copy()
    data_conv = data * photbw
    hdu.data = np.array(data_conv, dtype=np.float32)
    hdu.header['BUNIT'] = ('erg/s/cm2/pixel', '1e-20 erg/s/cm2')

    return(hdu)

hdu_hst_halpha = get_ergcm2sA_2_ergcm2s(hdu_hst_halpha, bp['%s_%s' %(instrument_f65Xn.upper(), narrowband_filter.upper())]['rectwidth'])
hdu_hst_cont = get_ergcm2sA_2_ergcm2s(hdu_hst_cont, bp['%s_%s' %(instrument_f65Xn.upper(), narrowband_filter.upper())]['rectwidth'])

# Saving .fits files

In [9]:
hdu_hst_f555w.writeto(rootdir+outputdir+'/hdu_hst_f547m.fits', overwrite=True)
hdu_hst_f65Xn.writeto(rootdir+outputdir+'/hdu_hst_%s.fits' %narrowband_filter, overwrite=True)
hdu_hst_f814w.writeto(rootdir+outputdir+'/hdu_hst_f814w.fits', overwrite=True)

hdu_hst_halpha.writeto(rootdir+outputdir+'/hdu_hst_%s_contsub.fits' %narrowband_filter, overwrite=True)

hdu_hst_cont.writeto(rootdir+outputdir+'/hdu_hst_%s_cont.fits' %narrowband_filter, overwrite=True)

hdu_muse['HA6562_FLUX'].writeto(rootdir+outputdir+'/hdu_muse_ha.fits', overwrite=True)